### EDA
EDA Goes into further detail
* notebook/Project_EDA_001_001.ipynb
* notebook/Project_EDA_002_001.ipynb

### Notebook with model training

notebook/Project_Model_Training.ipynb

Inside data_ingestion.py - src/components/data_ingestion.py

```python
import os
import sys
import pandas as pd

from sklearn.model_selection import train_test_split
from dataclasses import dataclass

from src.exception import CustomException
from src.logger import logging

@dataclass
class DataIngestionConfig:
    train_data_path: str = os.path.join('artifacts',"train.csv")
    test_data_path: str = os.path.join('artifacts',"test.csv")
    raw_data_path: str = os.path.join('artifacts', 'data.csv')

class DataIngestion:
    def __init__(self):
        self.ingestion_config = DataIngestionConfig()
    
    def initiate_data_ingestion(self):
        logging.info("Entered the data ingestion method or component")
        try:
            #pass
            df = pd.read_csv("notebooks/data/students_performance_0760.csv")
            logging.info("Reading the dataset as a DataFrame")
            
            os.makedirs(os.path.dirname(self.ingestion_config.train_data_path), exist_ok=True)
            
            df.to_csv(self.ingestion_config.raw_data_path, index=False, header=True)

            logging.info("Initiating Train Test Split")
            train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)
            logging.info("Train Test Split Complete")
            
            train_set.to_csv(self.ingestion_config.train_data_path, index=False, header=True)
            test_set.to_csv(self.ingestion_config.test_data_path, index=False, header=True)
            #logging.info("Train and Test Set Saved")
            logging.info("Data Ingestion Complete")
            return(
                self.ingestion_config.train_data_path,
                self.ingestion_config.test_data_path
            )
        except Exception as e:
            raise CustomException(e, sys)

if __name__ == "__main__":
    obj = DataIngestion()
    train_data, test_data = obj.initiate_data_ingestion()
```

Testing it

```Shell
% python src/components/data_ingestion.py
```

Inside logs
```Markdown
logs/04_07_2025_16_13_09.log/04_07_2025_16_13_09.log
[ 2025-04-07 16:13:09,068 ] 22 root - INFO - Entered the data ingestion method or component
[ 2025-04-07 16:13:09,074 ] 26 root - INFO - Reading the dataset as a DataFrame
[ 2025-04-07 16:13:09,076 ] 32 root - INFO - Initiating Train Test Split
[ 2025-04-07 16:13:09,077 ] 34 root - INFO - Train Test Split Completed
[ 2025-04-07 16:13:09,079 ] 38 root - INFO - Train and Test Set Saved
[ 2025-04-07 16:13:09,079 ] 39 root - INFO - Data Ingestion Completed

```
